# **Exploratory Data Analysis**:<a class="anchor" id="0"></a>


1. [**Data Import**](#1): Reading in the data

2. [**Initial Data Prep.**](#2): Run some joins and cleanup

3. [**Analysis**](#3): EDA

In [3]:
import numpy as np
import pandas as pd

from _util.custom_plotting import *
from _util.make_confusion_matrix import make_cm
from _util.model_comparisons import *
from _util.custom_mem_opt import custom_mem_opt

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
color = sns.color_palette()

root = './_data/'

## Data Import<a class=anchor id="1"></a>
[Back to top](#0)